In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

#Question 1
# Read the data in, can also use dtype guessing to handle the column memory demanding
# In this case, trun the low_memory = Flase to handle
train_data=pd.read_csv("data/Incidents_Responded_to_by_Fire_Companies.csv",low_memory=False)

# train_data=train_data.drop("Id",1)
train_data.head()

print('Total number of incidents: {}'.format(len(train_data)))

value = np.array(train_data['INCIDENT_TYPE_DESC'])
import collections
count = collections.Counter(value)
max_count_of_incident = list(count.most_common(1))
a = max_count_of_incident[0][1]/len(train_data)
print('The most common type of incident rate is {}'.format( a ))

#Question 2 
print('Ratio of 111-building fire to 651 - Smoke scare: {}'.format(count['111 - Building fire']/count['651 - Smoke scare, odor of smoke']))



Total number of incidents: 2277779
The most common type of incident rate is 0.3614828304238471
Ratio of 111-building fire to 651 - Smoke scare: 0.08789046762106846


In [12]:
# Question 3

a = train_data.loc[(train_data["INCIDENT_TYPE_DESC"]=="710 - Malicious, mischievous false call, other") & (train_data["BOROUGH_DESC"]=="1 - Manhattan") , ["INCIDENT_TYPE_DESC","BOROUGH_DESC"]]
b = train_data.loc[(train_data["INCIDENT_TYPE_DESC"]=="710 - Malicious, mischievous false call, other") & (train_data["BOROUGH_DESC"]=="3 - Staten Island") , ["INCIDENT_TYPE_DESC","BOROUGH_DESC"]]
print(a['BOROUGH_DESC'].count()/b['BOROUGH_DESC'].count())


3.0497825036249395


In [15]:
#Question 4

#train_data=pd.read_csv("data/Incidents_Responded_to_by_Fire_Companies.csv",low_memory=False)
Building_fire_start = train_data.loc[(train_data["INCIDENT_TYPE_DESC"]=="111 - Building fire") & (train_data["INCIDENT_DATE_TIME"]) , ["INCIDENT_TYPE_DESC","INCIDENT_DATE_TIME"]]
Building_fire_arrive = train_data.loc[(train_data["INCIDENT_TYPE_DESC"]=="111 - Building fire") & (train_data["ARRIVAL_DATE_TIME"]) , ["INCIDENT_TYPE_DESC","ARRIVAL_DATE_TIME"]]

Building_fire_start['INCIDENT_DATE_TIME'] = pd.to_datetime(Building_fire_start['INCIDENT_DATE_TIME'])
Building_fire_arrive['ARRIVAL_DATE_TIME'] = pd.to_datetime(Building_fire_arrive['ARRIVAL_DATE_TIME'])

Building_fire_arrive['Time_difference'] = Building_fire_arrive['ARRIVAL_DATE_TIME'] - Building_fire_start['INCIDENT_DATE_TIME']
Building_fire_arrive['Time_difference_second'] = Building_fire_arrive['Time_difference'].dt.total_seconds()

Building_fire_arrive['Time_difference_minutes'] = Building_fire_arrive['Time_difference_second'] / 60.0
output_quantile = Building_fire_arrive['Time_difference_minutes'].quantile([0.25,0.5,0.75])
print(output_quantile)


0.25    2.85
0.50    3.45
0.75    4.15
Name: Time_difference_minutes, dtype: float64


In [16]:
# Question 5 

Cooking_fire_and_time = train_data[["INCIDENT_TYPE_DESC" , "INCIDENT_DATE_TIME"]]
Cooking_fire_and_time['INCIDENT_DATE_TIME'] = pd.to_datetime(Cooking_fire_and_time['INCIDENT_DATE_TIME']).dt.hour

Cooking_fire_and_time
#=="113 - Cooking fire, confined to container"
a = Cooking_fire_and_time.loc[Cooking_fire_and_time['INCIDENT_TYPE_DESC'] =="113 - Cooking fire, confined to container"]
group1 = Cooking_fire_and_time.groupby('INCIDENT_DATE_TIME', as_index=False)['INCIDENT_TYPE_DESC'].count()

group2 = a.groupby('INCIDENT_DATE_TIME', as_index=False)['INCIDENT_TYPE_DESC'].count()

group3 = group2['INCIDENT_TYPE_DESC']/group1['INCIDENT_TYPE_DESC']
print('The highest cooking fire propotion is {}'.format(group3.max()))



The highest cooking fire propotion is 0.050846824501505385


C:\Users\Jie\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [19]:
# Question 6 
import numpy
resident_zip_code =pd.read_csv("data/2010+Census+Population+By+Zipcode+(ZCTA).csv",low_memory=False)
resident_zip_code["Zip Code ZCTA"] = resident_zip_code["Zip Code ZCTA"].astype('float')
resident_zip_code = resident_zip_code.rename(columns={"Zip Code ZCTA": "ZIP_CODE"})
zip_incident = train_data.loc[(train_data["INCIDENT_TYPE_DESC"]=="111 - Building fire") & (train_data["ZIP_CODE"]) , ["INCIDENT_TYPE_DESC","ZIP_CODE"]]
zip_incident["ZIP_CODE"] = zip_incident["ZIP_CODE"].astype(float)

population_zip_fire = pd.merge(zip_incident,resident_zip_code,how='inner', on = ['ZIP_CODE'] )

count_fire_with_pop = population_zip_fire.groupby(population_zip_fire['2010 Census Population']).size().reset_index(name='Num. Building fire')
count_fire_with_pop['2010 Census Population']

results = {}
coeffs = numpy.polyfit(count_fire_with_pop['2010 Census Population'],count_fire_with_pop['Num. Building fire'],2)
x = count_fire_with_pop['2010 Census Population']
y = count_fire_with_pop['Num. Building fire']
p = numpy.poly1d(coeffs)
# fit values, and mean
yhat = p(x)                         # or [p(z) for z in x]
ybar = numpy.sum(y)/len(y)          # or sum(y)/len(y)
ssreg = numpy.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
sstot = numpy.sum((y - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
results['determination'] = ssreg / sstot
print(results['determination'])



0.5943492242479399


In [20]:
# Question 7

CO_detector = train_data.loc[train_data["CO_DETECTOR_PRESENT_DESC"].isin(['Yes','No']) & train_data["CO_DETECTOR_PRESENT_DESC"]]
CO_detector = CO_detector[["CO_DETECTOR_PRESENT_DESC","INCIDENT_DATE_TIME","LAST_UNIT_CLEARED_DATE_TIME"]]
CO_detector['INCIDENT_DATE_TIME'] = pd.to_datetime(CO_detector['INCIDENT_DATE_TIME'])
CO_detector['LAST_UNIT_CLEARED_DATE_TIME'] = pd.to_datetime(CO_detector['LAST_UNIT_CLEARED_DATE_TIME'])

CO_detector['Time_difference'] = CO_detector['LAST_UNIT_CLEARED_DATE_TIME'] - CO_detector['INCIDENT_DATE_TIME']
CO_detector['Time_difference_minutes'] = CO_detector['Time_difference'].dt.total_seconds()/60.0

bins = [20,30,40,50,60,70]
group_names = ['20-30','30-40','40-50','50-60','60-70']

CO_detector["Time_difference_minutes"] = CO_detector["Time_difference_minutes"].astype('float')
CO_detector = CO_detector.loc[CO_detector["Time_difference_minutes"]>19.9999 ]
CO_detector = CO_detector.loc[CO_detector["Time_difference_minutes"]<70.001 ]

time_group_1 = CO_detector.groupby( pd.cut(CO_detector['Time_difference_minutes'],bins,include_lowest=True) ).count()
time_group_Yes = CO_detector.loc[CO_detector["CO_DETECTOR_PRESENT_DESC"] == 'Yes' ]

time_group_Yes = time_group_Yes.groupby( pd.cut(time_group_Yes['Time_difference_minutes'],bins,include_lowest=True) ).count()
time_group_No  = CO_detector.loc[CO_detector["CO_DETECTOR_PRESENT_DESC"] == 'No']

time_group_No = time_group_No.groupby( pd.cut(time_group_No['Time_difference_minutes'],bins,include_lowest=True) ).count()
a= time_group_No['CO_DETECTOR_PRESENT_DESC']/time_group_1['CO_DETECTOR_PRESENT_DESC']

b= time_group_Yes['INCIDENT_DATE_TIME']/time_group_1['INCIDENT_DATE_TIME']
a = np.array(a)

time_group_1 = time_group_1.rename(index=str,columns ={"CO_DETECTOR_PRESENT_DESC" : "count_total"})
time_group_1 = time_group_1["count_total"]
time_group_1 = np.array(time_group_1)

time_group_Yes  = time_group_Yes.rename(index=str,columns ={"CO_DETECTOR_PRESENT_DESC" : "count_yes"})
time_group_Yes = time_group_Yes["count_yes"]
time_group_Yes = np.array(time_group_Yes)

time_group_No  = time_group_No.rename(index=str,columns ={"CO_DETECTOR_PRESENT_DESC" : "count_no"})
time_group_No = time_group_No["count_no"]
time_group_No = np.array(time_group_No)
print(time_group_Yes,time_group_No)

number30 = CO_detector.loc[CO_detector["Time_difference_minutes"]==30.00 ]
number30 = number30['CO_DETECTOR_PRESENT_DESC'].count()
number40 = CO_detector.loc[CO_detector["Time_difference_minutes"]==40.00 ]
number40 = number40['CO_DETECTOR_PRESENT_DESC'].count()
number50 = CO_detector.loc[CO_detector["Time_difference_minutes"]==50.00 ]
number50 = number50['CO_DETECTOR_PRESENT_DESC'].count()
number60 = CO_detector.loc[CO_detector["Time_difference_minutes"]==60.00 ]
number60 = number60['CO_DETECTOR_PRESENT_DESC'].count()

num_add = [number30,number40,number50,number60]
print(time_group_1)

for i in range(1,len(time_group_1)-1):
    time_group_1[i] += num_add[i-1]

number30y = CO_detector.loc[CO_detector["Time_difference_minutes"]==30.00 ]
number30y = number30y.loc[number30y["CO_DETECTOR_PRESENT_DESC"] == 'Yes' ]
number30y = number30y['CO_DETECTOR_PRESENT_DESC'].count()

number40y = CO_detector.loc[CO_detector["Time_difference_minutes"]==40.00 ]
number40y = number40y.loc[number40y["CO_DETECTOR_PRESENT_DESC"] == 'Yes' ]
number40y = number40y['CO_DETECTOR_PRESENT_DESC'].count()

print(time_group_1)
time_group_Yes[1] += number30y
time_group_No[1] += num_add[0]-number30y

time_group_Yes[2] += number40y
time_group_No[2] += num_add[1]-number40y
print(time_group_Yes,time_group_No)

ratio_yes = time_group_Yes/time_group_1
ratio_no  = time_group_No/time_group_1
print(ratio_yes,ratio_no)

ratio = ratio_no/ratio_yes
print(ratio)

mid_point_bins = [25,35,45,55,65]
print(mid_point_bins)

p1 = numpy.polyfit (mid_point_bins,ratio,1)
result = numpy.polyval(p1,39)
print(result)


[6501 2145  923  481  235] [1181  586  348  249  158]
[7682 2731 1271  730  393]
[7682 2741 1276  730  393]
[6501 2155  926  481  235] [1181  586  350  249  158]
[0.84626399 0.78620941 0.72570533 0.65890411 0.59796438] [0.15373601 0.21379059 0.27429467 0.34109589 0.40203562]
[0.18166436 0.27192575 0.37796976 0.51767152 0.67234043]
[25, 35, 45, 55, 65]
0.3306884900414385


In [21]:
# Question 8

CO_detector_NO = train_data.loc[train_data["CO_DETECTOR_PRESENT_DESC"].isin(['No']) & train_data["CO_DETECTOR_PRESENT_DESC"]]
CO_detector_NO = CO_detector_NO[["CO_DETECTOR_PRESENT_DESC","INCIDENT_DATE_TIME","LAST_UNIT_CLEARED_DATE_TIME"]]
CO_detector_NO['INCIDENT_DATE_TIME'] = pd.to_datetime(CO_detector_NO['INCIDENT_DATE_TIME'])
CO_detector_NO['LAST_UNIT_CLEARED_DATE_TIME'] = pd.to_datetime(CO_detector_NO['LAST_UNIT_CLEARED_DATE_TIME'])
CO_detector_NO['Time_difference'] = CO_detector_NO['LAST_UNIT_CLEARED_DATE_TIME'] - CO_detector_NO['INCIDENT_DATE_TIME']
CO_detector_NO['Time_difference_minutes'] = CO_detector_NO['Time_difference'].dt.total_seconds()/60.0


small_than60 = CO_detector_NO.loc[CO_detector_NO["Time_difference_minutes"]<=60 ]
count_small60 = small_than60['CO_DETECTOR_PRESENT_DESC'].count()
print(count_small60)
over_than60 = CO_detector_NO.loc[CO_detector_NO["Time_difference_minutes"]>60 ]
over_small60 = over_than60['CO_DETECTOR_PRESENT_DESC'].count()
print(over_small60)

total = count_small60+ over_small60
pro1 = count_small60/total
pro2 = over_small60/total
print(pro1,pro2)

from scipy.stats import chisquare
chisq, p = chisquare([pro1,pro2],ddof=1)
print(chisq)


4785
859
0.8478029766123317 0.15219702338766833
0.4838676421615926
